# Example Games and Solutions

In [1]:
import numpy as np
from eqrpy.nash import Porter_Nudelman_Shoham as PNS
from eqrpy.correlated import ConvPolytope as CP

def pretty_print_solutions(sols, game_name, solution_type):
    if len(sols) == 0:
        print(f"No {solution_type} solution found in \033[1m{game_name}\033[0m game.")
    else:
        print(f"Found total {len(sols)} {solution_type} solution{'s' if len(sols) > 1 else ''} in \033[1m{game_name}\033[0m game:\n")
        if len(sols) <= 10:
            for i, s in enumerate(sols):
                print(f"strategies-({i+1}): {s[0]}")
                print(f"utilities-({i+1}): {s[1]}\n")
        else:
            print("Here shows only 10 example solutions:\n")
            for i, s in enumerate(sols[-10:]):
                print(f"strategies-({i+1}): {s[0]}")
                print(f"utilities-({i+1}): {s[1]}\n")

In [2]:
games = {}
games["chicken"] = {"nplayer":2, "nactions":[2,2], "payoff":[[[0,0],[7,2]], 
                                                             [[2,7],[6,6]]]}
games["crossroad"] = {"nplayer":2, "nactions":[2,2], "payoff":[[[-6,-6],[4,-4]], 
                                                               [[-4, 4],[2, 2]]]}
games["battle of sexes"] = {"nplayer":2, "nactions":[2,2], "payoff":[[[2,1],[-1,-1]], 
                                                                     [[-1,-1],[1,2]]]}
games["paper rock scissors"] = {"nplayer":2, "nactions":[3,3], "payoff":[[[0,0],[-1,1],[1,-1]],
                                                                         [[1,-1],[0,0],[-1,1]],
                                                                         [[-1,1],[1,-1],[0,0]]]}
games["prisoner's dilemma"] = {"nplayer":2, "nactions":[2,2], "payoff":[[[-2,-2],[-10,0]],
                                                                        [[0,-10],[-5,-5]]]}
games["match"] = {"nplayer":2, "nactions":[2,2], "payoff":[[[1,1],[0,0]],
                                                           [[0,0],[1,1]]]}
games["match2"] = {"nplayer":2, "nactions":[2,2], "payoff":[[[1,-1],[-1,1]],
                                                           [[-1,1],[1,-1]]]}
games["lazy cooperation"] = {"nplayer":3, "nactions":[3,3,3], "payoff":np.array([2,2,2]+[0]*36+[2,2,2]+[0]*36 +[2,2,2]).reshape(3,3,3,-1)}
games["Andreu Mas-Colell"] = {"nplayer":3, "nactions":[2,2,2], "payoff":[[[[1,1,-5],[-1,-1,5]],[[-5,-5,0],[-5,-5,0]]],
                                                                         [[[-5,-5,0],[-5,-5,0]],[[0,0,10],[-2,-2,0]]]]}
games["3 player matching pennies"] = {"nplayer":3, "nactions":[2,2,2], "payoff":[[[[1,1,-2],[-1,-1,2]],[[-1,-1,2],[-1,-1,2]]],
                                                                         [[[-1,-1,2],[-1,-1,2]],[[-1,-1,2],[1,1,-2]]]]}
games["Moreno and Wooders"] = {"nplayer":3, "nactions":[2,2,2], "payoff":[[[[3,2,0],[3,2,0]],[[0,0,0],[0,3,2]]],
                                                                         [[[2,0,3],[0,0,0]],[[2,0,3],[0,3,2]]]]}

## 1. Nash Equilibria
We denote a **normal-form game** with joint utility function $u$ as $\Gamma_u = (N, A, u)$.
 - $N = \{1,2,\cdots,n\}$ is the set of all $n$ players.
 - $A = A_1 \times A_2 \times \cdots \times A_n$ is the combinatorial action space of all players.
 - $u = (u_1, u_2, \cdots, u_n)$ is the utility function, where $u_i: A\rightarrow\mathbb R$.

We also define $\sigma = (\sigma_1, \sigma_2, \cdots, \sigma_n)$ as a mixed strategy profile, where $\sigma_i(a_i)$ specifies the probability of player $i$ choosing action $a_i\in A_i$, and $\sigma(a) := \prod_{i\in N}\sigma_i(a_i)$. The expected utility of player $i$ is given by $r_i(\sigma) := \sum_{a\in A} \sigma(a)u_i(a)$.

A mixed strategy profile $\sigma^*$ is a **Nash Equilibrium** (Nash, 1950), iff for each player $i\in N$, its strategy $\sigma_i^*$ is the best response given all other players' strategy $\sigma_{-i}^*$. I.e., $r_i(\sigma_i, \sigma_{-i}^*) \leq r_{i}(\sigma_i^*, \sigma_{-i}^*), \forall i\in N, \forall \sigma_i \in \Delta A_i$.

We solve mixed strategy NE using Porter-Nudelman-Shoham algorithm (2008):

NE: $\sum_{a\in A} [\sigma(a) u_i(a) - \sigma'_i(a_i)\sigma_{-i}(a_{-i}) u_i(a_i, a_{-i})] \geq 0, \forall i\in N, \forall \sigma'_i, \sigma=\prod_i\sigma_i$

$\Leftrightarrow$ $\sum_{a\in A} [\sigma(a) u_i(a) - \sigma_{-i}(a_{-i}) u_i(a_i', a_{-i})] \geq 0, \forall i\in N, \forall a_i'\in A_i, \sigma=\prod_i\sigma_i$

$\Leftrightarrow$ $\sum_{a\in A:a_i\in A} [ \sigma_{-i}(a_{-i}) u_i(a_i, a_{-i}) - \sigma_{-i}(a_{-i}) u_i(a_i', a_{-i})] \geq 0, \forall i\in N, \forall a_i'\in A_i, \forall a_i \text{ s.t. } \sigma_i(a_i)\neq 0, \sigma=\prod_i\sigma_i$

In [4]:
for game_name in games.keys():
    print("-"*100)
    solver = PNS(nplayer=games[game_name]["nplayer"], 
                 nactions=games[game_name]["nactions"], 
                 utilities=games[game_name]["payoff"])
    pretty_print_solutions(solver.solveNE(find_all=True), game_name=game_name, solution_type="NE")
    print("-"*100)

----------------------------------------------------------------------------------------------------
Found total 3 NE solutions in chicken game:

strategies-(1): [array([1., 0.]), array([0., 1.])]
utilities-(1): [7. 2.]

strategies-(2): [array([0., 1.]), array([1., 0.])]
utilities-(2): [2. 7.]

strategies-(3): [array([0.33333333, 0.66666667]), array([0.33333333, 0.66666667])]
utilities-(3): [4.66666667 4.66666667]

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Found total 3 NE solutions in crossroad game:

strategies-(1): [array([1., 0.]), array([0., 1.])]
utilities-(1): [ 4. -4.]

strategies-(2): [array([0., 1.]), array([1., 0.])]
utilities-(2): [-4.  4.]

strategies-(3): [array([0.5, 0.5]), array([0.5, 0.5])]
utilities-(3): [-1. -1.]

-------------------------------------------------------------------------------------------------

## 2. Refinement of Nash Equilibria
Nash Equilibrium assumes all players move independently. However, if players have **unlimited ability to pre-play communicate** and **reach non-binding agreements** regarding their strategy choices, they may form coalitions to reach **mutually beneficial agreements** and deviate from NE. Therefore, it's important to check if any strong enough NE exists where players have no incentive to deviate from such NE, before discussing CE and CommE.

### 2.1. Pareto-Optimal NEs
The first proposal for the strong enough NE is the Pareto-Optimal NE (PONE). For a strategy profile $\sigma \in \text{NEs}$ is Pareto-dominated by other solutions in NEs, then all players may try to avoid it via non-binding communication. However, there is no guarantee that players will pick any of these NEs, even though they must exist. Some subgroup of players may still have insentive to devitate from PNEs together. PNE is not strong enough.

In [5]:
for game_name in games.keys():
    print("-"*100)
    solver = PNS(nplayer=games[game_name]["nplayer"], 
                 nactions=games[game_name]["nactions"], 
                 utilities=games[game_name]["payoff"])
    pretty_print_solutions(solver.solvePONE(find_all=True), game_name=game_name, solution_type="Pareto-optimal NE")
    print("-"*100)

----------------------------------------------------------------------------------------------------
Found total 3 Pareto-optimal NE solutions in chicken game:

strategies-(1): [array([1., 0.]) array([0., 1.])]
utilities-(1): [7.000000000000001 2.0]

strategies-(2): [array([0., 1.]) array([1., 0.])]
utilities-(2): [2.0 7.0]

strategies-(3): [array([0.33333333, 0.66666667]) array([0.33333333, 0.66666667])]
utilities-(3): [4.666666666666667 4.666666666666667]

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Found total 3 Pareto-optimal NE solutions in crossroad game:

strategies-(1): [array([1., 0.]) array([0., 1.])]
utilities-(1): [4.0 -4.0]

strategies-(2): [array([0., 1.]) array([1., 0.])]
utilities-(2): [-4.0 4.0]

strategies-(3): [array([0.5, 0.5]) array([0.5, 0.5])]
utilities-(3): [-1.0 -1.0000000000000002]

----------------------

### 2.2. Strong NEs
The second proposal for the strong enough NE is the so-called "strong NE" (SNE, Aumann, 1959), which requires for all coalition $J\subseteq N$, there exists at least one player $i \in J$ such that $r_i(\sigma_{J}, \sigma^{*}_{-J}) \leq r_i(\sigma_{J}^*, \sigma^{*}_{-J})$. Thus no coalition can be effectively formed if SNEs exists. It's easy to see SNE $\subseteq$ NE ($|J|=1$) and SNE $\subseteq$ PONE ($J=N$). SNE is a too strong solution concept and rarely exits in non-cooperative games.   
Our implementation finds NEs that are resistant to pure multilateral deviation and deviation to other NEs, which only a necessary condition. A practical necessary and sufficient condition of SNE is still open. Please see a potential algorithm and more discussion in Gatti, Rocco and Sandholm.

In [6]:
for game_name in games.keys():
    print("-"*100)
    solver = PNS(nplayer=games[game_name]["nplayer"], 
                 nactions=games[game_name]["nactions"], 
                 utilities=games[game_name]["payoff"])
    pretty_print_solutions(solver.solveSNE(find_all=True), game_name=game_name, solution_type="strong NE (at most)")
    print("-"*100)

----------------------------------------------------------------------------------------------------
Found total 2 strong NE (at most) solutions in chicken game:

strategies-(1): [array([1., 0.]) array([0., 1.])]
utilities-(1): [7.000000000000001 2.0]

strategies-(2): [array([0., 1.]) array([1., 0.])]
utilities-(2): [2.0 7.0]

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Found total 2 strong NE (at most) solutions in crossroad game:

strategies-(1): [array([1., 0.]) array([0., 1.])]
utilities-(1): [4.0 -4.0]

strategies-(2): [array([0., 1.]) array([1., 0.])]
utilities-(2): [-4.0 4.0]

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Found total 2 strong NE (at most) solutions in ba

### 2.3 Coalition-Proof NEs
CPNE (Bernheim, Peleg, and Whinston, 1987) is a solution concept weaker than SNE. It defines self-enforcement of playing the solution strategy profile recursively:
1. In a single player game $\Gamma$, $\sigma^{*} \in \Delta(A_1)$ is a CPNE iff $r_1(\sigma^{*}) \geq r_1(\sigma)$ for all $\sigma \in \Delta(A_1)$.
2. Let $n > 1$ and assume that the CPNE has been defined for games with fewer than $n$ players. Then,  
 i). For any game $\Gamma$ with $n$ players, $\sigma^*$ is self-enforcing if, for all $J \subset \{1,\dots,n\}$, $\sigma_J^{*}$ is a CPNE in $\Gamma\backslash \sigma^{*}_{-J}$;   
 ii). For any game $\Gamma$ with $n$ players, $\sigma^*$ is CPNE if it's *self-enforcing* and not Pareto dominated by other *self-enforcing* $\sigma$.

Some observation is useful for design the algorithm. First, all CPNEs are NE ($|J|=1$). Second, for any subgames with 2 players, all self-enforcing strategies are NEs, and CPNE = PONE $\subseteq$ NE. 

In [7]:
for game_name in games.keys():
    print("-"*100)
    solver = PNS(nplayer=games[game_name]["nplayer"], 
                 nactions=games[game_name]["nactions"], 
                 utilities=games[game_name]["payoff"])
    pretty_print_solutions(solver.solveCPNE(find_all=True), game_name=game_name, solution_type="Coalition-Proof NE")
    print("-"*100)

----------------------------------------------------------------------------------------------------
Found total 3 Coalition-Proof NE solutions in chicken game:

strategies-(1): [array([1., 0.]) array([0., 1.])]
utilities-(1): [7.000000000000001 2.0]

strategies-(2): [array([0., 1.]) array([1., 0.])]
utilities-(2): [2.0 7.0]

strategies-(3): [array([0.33333333, 0.66666667]) array([0.33333333, 0.66666667])]
utilities-(3): [4.666666666666667 4.666666666666667]

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Found total 3 Coalition-Proof NE solutions in crossroad game:

strategies-(1): [array([1., 0.]) array([0., 1.])]
utilities-(1): [4.0 -4.0]

strategies-(2): [array([0., 1.]) array([1., 0.])]
utilities-(2): [-4.0 4.0]

strategies-(3): [array([0.5, 0.5]) array([0.5, 0.5])]
utilities-(3): [-1.0 -1.0000000000000002]

--------------------

## 3. Correlated Equilibria

Nash Equilibrium assumes all players move independently. However, if every player $i$ can make a private observation $o_i\in O_i$ of the value of some public signal $\omega \in \Omega$, their strategies can be correlated. For example, in the case where players can have pairwise communication, $\Omega$ is a set of all possible messages $\{(m_{j\rightarrow k})_{j\neq k}\}$, and each individual's observation only contains messages involve itself $o_i(\omega) = (m_{j\rightarrow i}, m_{i\rightarrow j})_{j\neq i}$.

Formally, we denote the game with partially observable public signal an **imperfect information game** $\Gamma_{u,d,p} = (N, A, u, \Omega, O, d, p)$,
 - $\Omega$ is the set of all possible public signals.
 - $O = O_1\times O_2 \times \cdots \times O_n$ is the combinatorial observation space of all players.
 - $d \in \Delta \Omega$ is the prior of public signal.
 - $p = (p_1, p_2, \cdots, p_n)$ is the observation likelihood, where $p_i(o_i|\omega)$ gives the probability the player $i$ observes $o_i$ when the public signal is $\omega$, and the joint likelihood is $p(o|\omega) := \prod_{i\in N}p_i(o_i|\omega)$.

Any player $i$ knowns the posterior of the public signal when observing $o_i$, which is calculated by $q_i(\omega|o_i) := p_i(o_i|\omega) d(\omega) / \sum_{\omega\in\Omega} p_i(o_i|\omega) d(\omega)$. 

Everyone's **underlying plan**, $s_i$, is a function of the public signal, $s_i(a_i|\omega)$, specifying the probability of player $i$ choosing action $a_i\in A_i$ when it knows/believes the public signal is $\omega$. The **reactive strategy** of a player $i$ is denoted by $\sigma_i(a_i|o_i)$. We also define $\sigma(a|o) := \prod_{i\in N} \sigma(a_i|o_i)$. Note that generally, there's no obvious relationship between the underlying plan and reactive strategy. Sometimes a play may adopt the average plan based on the posterior as its reaction strategy, $\sigma_i(a_i|o_i)= \sum_{\omega'\in\Omega}s_i(a_i|\omega')q_i(\omega'|o_i)$.

Moreover, the **signaled strategy** of player $i$ given by $\sigma_i(a_i|\omega) := \sum_{o_i\in O_i} \sigma_i(a_i|o_i) p_i(o_i|\omega)$, when the public signal is $\omega$. Finally, we can define **marginalized strategy** as $\sigma_i(a_i) = \sum_{\omega\in\Omega}\sigma(a_i | \omega)d(\omega)$. In this signaled game, players' actions are non-independent when the public signal is unknown, $\sigma(a) = \sum_{\omega\in \Omega}d(\omega)\prod_{i\in N}\sigma_i(a_i|\omega) \neq \prod_{i\in N}\sigma_i(a_i) = \prod_{i\in N}\sum_{\omega\in \Omega}\sigma_i(a_i|\omega)d(\omega)$. When observing $o_i$ and everyone following the signaled strategy profile $\sigma$, the the expected utility of player $i$ is given by $r_i(\sigma | o_i) = \sum_{\omega\in \Omega}\sum_{a\in A}q_i(\omega|o_i)\sigma(a|\omega)u_i(a)$.

A signaled strategy profile $\sigma^*$ is called **Correlated Equilibrium** (Aumann, 1974), iff for each player $i\in N$, it has no incentive to deviate from its signaled strategy $\sigma^*_{i}$. I.e., $r_i(\sigma_i, \sigma_{-i}^* | o_i) \leq r_i(\sigma_i^*, \sigma_{-i}^* | o_i), \forall i \in N, \forall \sigma_i \in (\Delta A_i)^{\Omega}$. Note that a signaled strategy $\sigma(a|\omega)$ and observation likelihoods $\{p_i\}$ can uniquely determine a reactive strategy $\sigma(a|o)$.

Papadimitriou and Roughgarden (2008) argued that CEs (marginalized strategies $\sigma(a)$ in our terminology) can be solved in polynomial time via linear programming ("Ellipsoid Against Hope"), if there is a polynomial seperation oracle. Their algorithm outputs CE as convex combinations of polynomial numbers of product probabilities. However, Stein, Parrilo and Ozdaglar (2010) showed that the ellipsoid-against-hope algorithm by can fail to find exact correl ted equilibria in certain cases. Jiang and Leyton-Brown (2010) modifed the use of the separation oracle to compute exact CEs in polynomial time.

- NE: $\sum_{a\in A:a_i \in a, \sigma_i(a_i)\neq 0} [\sigma_{-i}(a_{-i}) u_i(a) - \sigma_{-i}(a_{-i}) u_i(a_i', a_{-i})] \geq 0, \forall i\in N,  \forall a_i'\in A_i$
- CE: $\sum_{a\in A: a_i \in a} \sigma(a) [u_i(a_i, a_{-i}) - u_i(a_i', a_{-i})] \geq 0, \forall i\in N, \forall a_i, a_i'\in A_i$
- Coarse CE: $\sum_{a\in A} \sigma(a) [u_i(a_i, a_{-i}) - u_i(a_i', a_{-i})] \geq 0, \forall i\in N, \forall a_i'\in A_i$  

It's easy to see SNE $\subseteq$ CPNE $\subseteq$ NE $\subseteq$ CE $\subseteq$ CCE.

In [8]:
for game_name in games.keys():
    print("-"*100)
    solver = CP(nplayer=games[game_name]["nplayer"], 
                 nactions=games[game_name]["nactions"], 
                 utilities=games[game_name]["payoff"])
    pretty_print_solutions(solver.solveCE(backend='ppl'), game_name=game_name, solution_type="CE basis")
    print("-"*100)

----------------------------------------------------------------------------------------------------
Found total 5 CE basis solutions in chicken game:

strategies-(1): [0. 0. 1. 0.]
utilities-(1): [2. 7.]

strategies-(2): [0. 1. 0. 0.]
utilities-(2): [7. 2.]

strategies-(3): [0.11111111 0.22222222 0.22222222 0.44444444]
utilities-(3): [4.66666667 4.66666667]

strategies-(4): [0.2 0.4 0.4 0. ]
utilities-(4): [3.6 3.6]

strategies-(5): [0.   0.25 0.25 0.5 ]
utilities-(5): [5.25 5.25]

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Found total 5 CE basis solutions in crossroad game:

strategies-(1): [0. 0. 1. 0.]
utilities-(1): [-4.  4.]

strategies-(2): [0. 1. 0. 0.]
utilities-(2): [ 4. -4.]

strategies-(3): [0.25 0.25 0.25 0.25]
utilities-(3): [-1. -1.]

strategies-(4): [0.33333333 0.33333333 0.33333333 0.        ]
utilities-(4): [-2.

### 3.1. Selected Correlated Equilibria (Maximal Sum of Utilities)

In [9]:
import warnings
from scipy.linalg import LinAlgWarning
warnings.filterwarnings(action='ignore', category=LinAlgWarning)

for game_name in games.keys():
    print("-"*100)
    solver = CP(nplayer=games[game_name]["nplayer"], 
                 nactions=games[game_name]["nactions"], 
                 utilities=games[game_name]["payoff"])
    pretty_print_solutions(solver.solveCE(final_special=[[1]*games[game_name]["nplayer"]], backend='ppl'), game_name=game_name, solution_type="maximal-total-untility CE")
    print("-"*100)

----------------------------------------------------------------------------------------------------
Found total 1 maximal-total-untility CE solution in chicken game:

strategies-(1): [8.29359599e-10 2.50000003e-01 2.50000003e-01 5.00000003e-01]
utilities-(1): [5.25000005 5.25000005]

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Found total 1 maximal-total-untility CE solution in crossroad game:

strategies-(1): [5.56601074e-11 3.33333333e-01 3.33333333e-01 3.33333334e-01]
utilities-(1): [0.66666667 0.66666667]

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Found total 1 maximal-total-untility CE solution in battle of sexes game:

strategies-(1): [5.00000000e-01 2.61707888e-13 1

## 3.2. Pareto-Optimal Correlated Equilibria
Similar to PONE, POCE is defined by the Pareto-frontier of all CEs. Since all CEs are in a convex polytope, any convex combination of basis CEs is an CE, and the utility is the same convex combination of basis utilities. Therefore, all CE utilies is also a convex polytope. All POCEs lie on the boundary of convex hull of basis CE utilies.

In [10]:
for game_name in games.keys():
    print("-"*100)
    solver = CP(nplayer=games[game_name]["nplayer"], 
                nactions=games[game_name]["nactions"], 
                utilities=games[game_name]["payoff"])
    pretty_print_solutions(solver.solvePOCE(backend='ppl'), game_name=game_name, solution_type="POCE basis")
    print("-"*100)

----------------------------------------------------------------------------------------------------
Found total 3 POCE basis solutions in chicken game:

strategies-(1): [0. 0. 1. 0.]
utilities-(1): [2. 7.]

strategies-(2): [0. 1. 0. 0.]
utilities-(2): [7. 2.]

strategies-(3): [0.   0.25 0.25 0.5 ]
utilities-(3): [5.25 5.25]

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Found total 3 POCE basis solutions in crossroad game:

strategies-(1): [0. 0. 1. 0.]
utilities-(1): [-4.  4.]

strategies-(2): [0. 1. 0. 0.]
utilities-(2): [ 4. -4.]

strategies-(3): [0.         0.33333333 0.33333333 0.33333333]
utilities-(3): [0.66666667 0.66666667]

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------